In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision import datasets, models, transforms
import os
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torch.autograd import Variable
import time

In [3]:
data_dir = 'F:\DataSets/EmotionImages/TargetData'

data_transform = {x: transforms.Compose([transforms.Resize([224, 224]),  # 设置尺寸
                                        transforms.ToTensor(),  # 转为Tensor
                                        transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])  # 标准化
                  for x in {"train", "valid"}}  # {"train":"训练集数据格式","valid":"测试集数据格式"}

image_datasets = {x: datasets.ImageFolder(root=os.path.join(data_dir, x),  # 载入数据
                                         transform = data_transform[x])
                  for x in {"train", "valid"}}  # {"train":"训练集","valid":"测试集"}

dataloader = {x: torch.utils.data.DataLoader(dataset=image_datasets[x],
                                            batch_size=16,
                                            shuffle=True)
              for x in {"train", "valid"}}  # {包装16个为一个批次"train":"训练集数据载入","valid":"测试集数据载入"}

In [8]:
print(len(dataloader["train"]))
print(len(dataloader["valid"]))

116
15


In [4]:
X_example, y_example = next(iter(dataloader["train"]))  # 迭代得到一个批次的样本
example_classes = image_datasets["train"].classes
index_classes = image_datasets["train"].class_to_idx
print(len(example_classes))
print((len(index_classes)))

3
3


In [17]:
# model = models.vgg16(pretrained=True)  # 使用VGG16 网络预训练好的模型

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\WZQAdmin/.cache\torch\checkpoints\vgg16-397923af.pth
100%|███████████████████████████████████████████████████████████████████████████████| 528M/528M [01:05<00:00, 8.45MB/s]


In [2]:
model = torch.load('F:\DataSets/VGG16.pth')

In [20]:
for parma in model.parameters():  # 设置自动梯度为false
    parma.requires_grad = False

model.classifier = torch.nn.Sequential(  # 修改全连接层 自动梯度会恢复为默认值
    torch.nn.Linear(25088, 4096),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(4096, 4096),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(4096, 3))
Use_gpu = torch.cuda.is_available()
if Use_gpu:  # 判断是否有cuda
    model = model.cuda()

loss_f = torch.nn.CrossEntropyLoss()  # 设置残差损失
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.00001)  # 使用Adam优化函数

epoch_n = 5
time_open = time.time()

In [22]:
for epoch in range(epoch_n):
    print("Epoch{}/{}".format(epoch,epoch_n-1))
    print("-"*10)
    for phase in {"train","valid"}:
        if phase == "train":
            print("Training...")
            model.train(True)
        else:
            print("Validing...")
            model.train(False)

        running_loss = 0.0
        running_corrects = 0
        for batch, data in enumerate(dataloader[phase], 1):  # enumerate 得到下标和数据
            X, y = data
            if Use_gpu:
                X, y = Variable(X.cuda()), Variable(y.cuda())  # **************************************
            else:
                X, y = Variable(X), Variable(y)
            y_pred = model(X)  # 预测
            _, pred = torch.max(y_pred, 1)
            optimizer.zero_grad()  # 梯度归零
            loss = loss_f(y_pred, y)  # 设置损失

            if phase == "train":
                loss.backward()  # 反向传播
                optimizer.step()  # 更新参数
            running_loss += loss.item()
            running_corrects += torch.sum(pred == y.data)

            if batch % 10 == 0 and phase == "train":
                print("Batch{},TrainLoss:{:.4f},Train ACC:{:.4f}".format(
                    batch, running_loss / batch, 100 * running_corrects / (16 * batch)))
        epocn_loss = running_loss * 16 / len(image_datasets[phase])
        epoch_acc = 100 * running_corrects / len(image_datasets[phase])
        print("{} Loss:{:.4f} Acc:{:4f}%".format(phase, epocn_loss, epoch_acc))
time_end = time.time() - time_open
print(time_end)

Epoch0/4
----------
Training...
Batch10,TrainLoss:0.8860,Train ACC:65.0000
Batch20,TrainLoss:0.8407,Train ACC:67.0000
Batch30,TrainLoss:0.8023,Train ACC:71.0000
Batch40,TrainLoss:0.7828,Train ACC:72.0000
Batch50,TrainLoss:0.7583,Train ACC:73.0000
Batch60,TrainLoss:0.7255,Train ACC:74.0000
Batch70,TrainLoss:0.7096,Train ACC:75.0000
Batch80,TrainLoss:0.6872,Train ACC:76.0000
Batch90,TrainLoss:0.6715,Train ACC:76.0000
Batch100,TrainLoss:0.6643,Train ACC:76.0000
Batch110,TrainLoss:0.6518,Train ACC:76.0000
train Loss:0.6437 Acc:77.000000%
Validing...
valid Loss:0.5886 Acc:73.000000%
Epoch1/4
----------
Training...
Batch10,TrainLoss:0.4136,Train ACC:83.0000
Batch20,TrainLoss:0.3924,Train ACC:85.0000
Batch30,TrainLoss:0.3772,Train ACC:87.0000
Batch40,TrainLoss:0.3673,Train ACC:88.0000
Batch50,TrainLoss:0.3653,Train ACC:88.0000
Batch60,TrainLoss:0.3617,Train ACC:87.0000
Batch70,TrainLoss:0.3657,Train ACC:87.0000
Batch80,TrainLoss:0.3578,Train ACC:88.0000
Batch90,TrainLoss:0.3510,Train ACC:88.0

In [28]:
torch.save(model, 'F:\DataSets/VGG16.pth')